In [31]:
import pandas as pd
import os
from glob import glob
from collections import defaultdict
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import re

In [32]:
# Funzione per aprire i file in una directory e ordinarli
def open_files(directory):
    files = os.listdir(directory)
    files.sort()
    return files

In [33]:
def round_sig(x, sig=3):
    if isinstance(x, (float, int)):
        return float(f"{x:.{sig}g}")
    return x

def format_report(report_dict):
    rounded = {}
    for label, metrics in report_dict.items():
        if isinstance(metrics, dict):
            rounded[label] = {k: round_sig(v) for k, v in metrics.items()}
        else:
            rounded[label] = round_sig(metrics)
    return rounded

def print_formatted_report(report_dict):
    print("\n📈 Report di classificazione (precision, recall, f1-score, support):")
    labels = [label for label in report_dict if label not in ('accuracy', 'macro avg', 'weighted avg')]
    header = f"{'Label':<20} {'Prec':>8} {'Rec':>8} {'F1':>8} {'Support':>8}"
    print(header)
    print("-" * len(header))
    for label in labels + ['macro avg', 'weighted avg']:
        row = report_dict[label]
        print(f"{label:<20} {row['precision']:>8.3f} {row['recall']:>8.3f} {row['f1-score']:>8.3f} {row['support']:>8.0f}")
    print(f"{'Accuracy':<20} {'':>8} {'':>8} {'':>8} {report_dict['accuracy']:>8.3f}")

def calculate_metrics(df, print_confusion=False):
    # Assumiamo che prediction e actual siano le colonne corrette
    df['extracted_prediction'] = df['prediction'].astype(str)
    df['rhetorical_figure'] = df['actual'].astype(str)  # assicurati che 'actual' sia la colonna giusta
    
    # Rimuoviamo eventuali righe con valori mancanti nelle due colonne (evita errori)
    df_clean = df

    if print_confusion:
        print(f"\n🧩 Matrice di confusione:")
        confusion = pd.crosstab(df_clean['rhetorical_figure'], df_clean['extracted_prediction'],
                                rownames=['Actual'], colnames=['Predicted'])
        print(confusion)

    report = classification_report(df_clean['rhetorical_figure'], df_clean['extracted_prediction'], output_dict=True, zero_division=0)
    return report

def average_reports(reports):
    avg_report = {}
    keys = reports[0].keys()

    for key in keys:
        if isinstance(reports[0][key], dict):
            avg_report[key] = {}
            for metric in reports[0][key]:
                values = [r[key].get(metric, 0.0) for r in reports if key in r]
                avg_report[key][metric] = np.mean(values)
        else:  # accuracy
            values = [r.get(key, 0.0) for r in reports]
            avg_report[key] = np.mean(values)

    return avg_report


In [34]:
model_files = open_files('models_generations')
actual_files = open_files('actuals')

models = {}
for model_file in model_files:
    # estrai il nome modello rimuovendo pattern specifici
    model_name = re.sub(r"^(fine-tuned-)?|-decoding-\d+\.csv$", "", model_file)
    print(f"Processing model: {model_name}")

    model = pd.read_csv(os.path.join('models_generations', model_file))

    reports = []
    for actual_file in actual_files:
        actual = pd.read_csv(os.path.join('actuals', actual_file))

        # fai il join di actual e model, utilizzando la colonna "pippo" di actual e l'index di model
        merged = pd.merge(actual, model, left_on='index', right_index=True, how='left')

        print(f"\n📊 Report per il modello '{model_name}' con file '{actual_file}':")
        report = calculate_metrics(merged, print_confusion=False)
        #print(report)
        formatted_report = format_report(report)
        print_formatted_report(formatted_report)

Processing model: LLaMAntino-3-ANITA-8B-Inst-DPO-ITA_predictions.csv

📊 Report per il modello 'LLaMAntino-3-ANITA-8B-Inst-DPO-ITA_predictions.csv' con file 'female_actual_labels.csv':

📈 Report di classificazione (precision, recall, f1-score, support):
Label                    Prec      Rec       F1  Support
--------------------------------------------------------
ANALOGY                 0.261    0.500    0.343       12
CONTEXT SHIFT           0.438    0.269    0.333       26
EUPHEMISM               0.500    0.200    0.286        5
FALSE ASSERTION         0.333    0.111    0.167        9
HYPERBOLE               0.500    0.167    0.250        6
OTHER                   0.385    0.417    0.400       12
OXYMORON                0.067    0.500    0.118        2
RHETORICAL QUESTION     0.625    0.500    0.556       10
macro avg               0.388    0.333    0.306       82
weighted avg            0.415    0.329    0.339       82
Accuracy                                           0.329

📊 Rep